# Tuning Neural Networks with Regularization - Lab 

## Introduction

In this lab, you'll use a train-test partition as well as a validation set to get better insights about how to tune neural networks using regularization techniques. You'll start by repeating the process from the last section: importing the data and performing preprocessing including one-hot encoding. From there, you'll define and compile the model like before. 

## Objectives

You will be able to:

- Apply early stopping criteria with a neural network 
- Apply L1, L2, and dropout regularization on a neural network  
- Examine the effects of training with more data on a neural network  


## Load the Data

Run the following cell to import some of the libraries and classes you'll need in this lab. 

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.text import Tokenizer

import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

The data is stored in the file `'Bank_complaints.csv'`. Load and preview the dataset.

In [2]:
# Load and preview the dataset
df = pd.read_csv('Bank_complaints.csv')
df.head()

,Product,Consumer complaint narrative
0,Student loan,In XX/XX/XXXX I filled out the Fedlaon applica...
1,Student loan,I am being contacted by a debt collector for p...
2,Student loan,I cosigned XXXX student loans at SallieMae for...
3,Student loan,Navient has sytematically and illegally failed...
4,Student loan,My wife became eligible for XXXX Loan Forgiven...


## Preprocessing Overview

Before you begin to practice some of your new tools such as regularization and optimization, let's practice munging some data as you did in the previous section with bank complaints. Recall some techniques:

* Sampling in order to reduce training time (investigate model accuracy vs data size later on)
* Train - test split
* One-hot encoding your complaint text
* Transforming your category labels 

## Preprocessing: Generate a Random Sample

Since you have quite a bit of data and training neural networks takes a substantial amount of time and resources, downsample in order to test your initial pipeline. Going forward, these can be interesting areas of investigation: how does your model's performance change as you increase (or decrease) the size of your dataset?  

- Generate a random sample of 10,000 observations using seed 123 for consistency of results. 
- Split this sample into `X` and `y` 

In [3]:
# Downsample the data
df_sample = df.sample(10000, random_state=123)

# Split the data into X and y
y = df_sample['Product']
X = df_sample['Consumer complaint narrative']

## Train-test split

- Split the data into training and test sets 
- Assign 1500 obervations to the test set and use 42 as the seed 

In [4]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1500, random_state = 42)

## Validation set 

As mentioned in the previous lesson, it is good practice to set aside a validation set, which is then used during hyperparameter tuning. Afterwards, when you have decided upon a final model, the test set can then be used to determine an unbiased perforance of the model. 

Run the cell below to further divide the training data into training and validation sets. 

In [5]:
# Split the data into training and validation sets
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train, y_train, test_size=1000, random_state=42)

## Preprocessing: One-hot Encoding the Complaints

As before, you need to do some preprocessing before building a neural network model. 

- Keep the 2,000 most common words and use one-hot encoding to reformat the complaints into a matrix of vectors 
- Transform the training, validate, and test sets 

In [6]:
# Use one-hot encoding to reformat the complaints into a matrix of vectors 
# Only keep the 2000 most common words 

tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(X_train_final)


X_train_tokens = tokenizer.texts_to_matrix(X_train_final, mode='binary')
X_val_tokens = tokenizer.texts_to_matrix(X_val, mode='binary')
X_test_tokens = tokenizer.texts_to_matrix(X_test, mode='binary')

## Preprocessing: Encoding the Products

Similarly, now transform the descriptive product labels to integers labels. After transforming them to integer labels, retransform them into a matrix of binary flags, one for each of the various product labels.  
  
> **Note**: This is similar to your previous work with dummy variables. Each of the various product categories will be its own column, and each observation will be a row. In turn, each of these observation rows will have a 1 in the column associated with it's label, and all other entries for the row will be zero. 

Transform the training, validate, and test sets. 

In [7]:
# Transform the product labels to numerical values
lb = LabelBinarizer()
lb.fit(y_train_final)

y_train_lb = to_categorical(lb.transform(y_train_final))[:, :, 1]
y_val_lb = to_categorical(lb.transform(y_val))[:, :, 1]
y_test_lb = to_categorical(lb.transform(y_test))[:, :, 1]

## A Baseline Model 

Rebuild a fully connected (Dense) layer network:  
- Use 2 hidden layers with 50 units in the first and 25 in the second layer, both with `'relu'` activation functions (since you are dealing with a multiclass problem, classifying the complaints into 7 classes) 
- Use a `'softmax'` activation function for the output layer  

In [11]:
# Build a baseline neural network model using Keras
random.seed(123)
from keras import models
from keras import layers
baseline_model = models.Sequential()

baseline_model.add(layers.Dense(units = 50, activation = 'relu', input_shape = (2000,)))
baseline_model.add(layers.Dense(units = 25, activation = 'relu'))
baseline_model.add(layers.Dense(units = 7 , activation = 'softmax'))


### Compile the Model

Compile this model with: 

- a stochastic gradient descent optimizer 
- `'categorical_crossentropy'` as the loss function 
- a focus on `'accuracy'` 

In [12]:
# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='categorical_crossentropy', 
                       metrics=['accuracy'])

### Train the Model

- Train the model for 150 epochs in mini-batches of 256 samples 
- Include the `validation_data` argument to ensure you keep track of the validation loss  

In [13]:
# Train the model
baseline_model_val = baseline_model.fit(X_train_tokens, 
                                        y_train_lb, 
                                        epochs=150, 
                                        batch_size=256, 
                                        validation_data=(X_val_tokens, y_val_lb))

Epoch 1/150
30/30 [==============================] - 1s 32ms/step - loss: 1.9617 - accuracy: 0.1391 - val_loss: 1.9452 - val_accuracy: 0.1700
Epoch 2/150
30/30 [==============================] - 0s 12ms/step - loss: 1.9386 - accuracy: 0.1701 - val_loss: 1.9302 - val_accuracy: 0.1760
Epoch 3/150
30/30 [==============================] - 0s 13ms/step - loss: 1.9233 - accuracy: 0.1973 - val_loss: 1.9176 - val_accuracy: 0.2020
Epoch 4/150
30/30 [==============================] - 0s 13ms/step - loss: 1.9096 - accuracy: 0.2139 - val_loss: 1.9050 - val_accuracy: 0.2170
Epoch 5/150
30/30 [==============================] - 0s 13ms/step - loss: 1.8954 - accuracy: 0.2335 - val_loss: 1.8917 - val_accuracy: 0.2440
Epoch 6/150
30/30 [==============================] - 0s 13ms/step - loss: 1.8801 - accuracy: 0.2520 - val_loss: 1.8769 - val_accuracy: 0.2550
Epoch 7/150
30/30 [==============================] - 0s 13ms/step - loss: 1.8628 - accuracy: 0.2671 - val_loss: 1.8596 - val_accuracy: 0.2700
Epoch 

30/30 [==============================] - 0s 12ms/step - loss: 0.5781 - accuracy: 0.8051 - val_loss: 0.6870 - val_accuracy: 0.7380
Epoch 59/150
30/30 [==============================] - 0s 12ms/step - loss: 0.5724 - accuracy: 0.8084 - val_loss: 0.6835 - val_accuracy: 0.7400
Epoch 60/150
30/30 [==============================] - 0s 11ms/step - loss: 0.5661 - accuracy: 0.8087 - val_loss: 0.6879 - val_accuracy: 0.7350
Epoch 61/150
30/30 [==============================] - 0s 11ms/step - loss: 0.5610 - accuracy: 0.8105 - val_loss: 0.6793 - val_accuracy: 0.7300
Epoch 62/150
30/30 [==============================] - 0s 12ms/step - loss: 0.5552 - accuracy: 0.8137 - val_loss: 0.6787 - val_accuracy: 0.7360
Epoch 63/150
30/30 [==============================] - 0s 12ms/step - loss: 0.5498 - accuracy: 0.8195 - val_loss: 0.6746 - val_accuracy: 0.7370
Epoch 64/150
30/30 [==============================] - 0s 12ms/step - loss: 0.5447 - accuracy: 0.8177 - val_loss: 0.6707 - val_accuracy: 0.7470
Epoch 65/150

30/30 [==============================] - 0s 12ms/step - loss: 0.3602 - accuracy: 0.8900 - val_loss: 0.6598 - val_accuracy: 0.7540
Epoch 116/150
30/30 [==============================] - 0s 11ms/step - loss: 0.3574 - accuracy: 0.8904 - val_loss: 0.6540 - val_accuracy: 0.7470
Epoch 117/150
30/30 [==============================] - 0s 11ms/step - loss: 0.3550 - accuracy: 0.8913 - val_loss: 0.6607 - val_accuracy: 0.7470
Epoch 118/150
30/30 [==============================] - 0s 11ms/step - loss: 0.3525 - accuracy: 0.8920 - val_loss: 0.6553 - val_accuracy: 0.7470
Epoch 119/150
30/30 [==============================] - 0s 12ms/step - loss: 0.3498 - accuracy: 0.8940 - val_loss: 0.6643 - val_accuracy: 0.7520
Epoch 120/150
30/30 [==============================] - 0s 12ms/step - loss: 0.3470 - accuracy: 0.8951 - val_loss: 0.6592 - val_accuracy: 0.7520
Epoch 121/150
30/30 [==============================] - 0s 11ms/step - loss: 0.3440 - accuracy: 0.8975 - val_loss: 0.6589 - val_accuracy: 0.7460
Epoch 

### Model Performance

The attribute `.history` (stored as a dictionary) contains four entries now: one per metric that was being monitored during training and validation. Print the keys of this dictionary for confirmation: 

In [14]:
# Access the history attribute and store the dictionary
baseline_model_val_dict = baseline_model_val.history

# Print the keys
baseline_model_val_dict.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

Evaluate this model on the training data: 

In [15]:
results_train = baseline_model.evaluate(X_train_tokens, y_train_lb)
print('----------')
print(f'Training Loss: {results_train[0]:.3} \nTraining Accuracy: {results_train[1]:.3}')

235/235 [==============================] - 1s 3ms/step - loss: 0.2750 - accuracy: 0.9251
----------
Training Loss: 0.275 
Training Accuracy: 0.925


Evaluate this model on the test data: 

In [16]:
results_test = baseline_model.evaluate(X_test_tokens, y_test_lb)
print('----------')
print(f'Test Loss: {results_test[0]:.3} \nTest Accuracy: {results_test[1]:.3}')

47/47 [==============================] - 0s 3ms/step - loss: 0.6153 - accuracy: 0.7840
----------
Test Loss: 0.615 
Test Accuracy: 0.784


### Plot the Results 

Plot the loss versus the number of epochs. Be sure to include the training and the validation loss in the same plot. 

In [ ]:
# Loss vs number of epochs with train and validation sets

Create a second plot comparing training and validation accuracy to the number of epochs. 

In [ ]:
# Accuracy vs number of epochs with train and validation sets

Did you notice an interesting pattern here? Although the training accuracy keeps increasing when going through more epochs, and the training loss keeps decreasing, the validation accuracy and loss don't necessarily do the same. After a certain point, validation accuracy keeps swinging, which means that you're probably **overfitting** the model to the training data when you train for many epochs past a certain dropoff point. Let's tackle this now. You will now specify an early stopping point when training your model. 


## Early Stopping

Overfitting neural networks is something you **_want_** to avoid at all costs. However, it's not possible to know in advance how many *epochs* you need to train your model on, and running the model multiple times with varying number of *epochs* maybe helpful, but is a time-consuming process. 

We've defined a model with the same architecture as above. This time specify an early stopping point when training the model. 

In [17]:
random.seed(123)
model_2 = models.Sequential()
model_2.add(layers.Dense(50, activation='relu', input_shape=(2000,)))
model_2.add(layers.Dense(25, activation='relu'))
model_2.add(layers.Dense(7, activation='softmax'))

model_2.compile(optimizer='SGD', 
                loss='categorical_crossentropy', 
                metrics=['acc'])

- Import `EarlyStopping` and `ModelCheckpoint` from `keras.callbacks` 
- Define a list, `early_stopping`: 
  - Monitor `'val_loss'` and continue training for 10 epochs before stopping 
  - Save the best model while monitoring `'val_loss'` 
 
> If you need help, consult [documentation](https://keras.io/callbacks/).   

In [18]:
# Import EarlyStopping and ModelCheckpoint
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Define the callbacks
early_stopping = [EarlyStopping(monitor='val_loss', patience=10), 
                  ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

Train `model_2`. Make sure you set the `callbacks` argument to `early_stopping`. 

In [19]:
model_2_val = baseline_model.fit(X_train_tokens, 
                                        y_train_lb, 
                                        epochs=150, 
                                        batch_size=256, 
                                        validation_data=(X_val_tokens, y_val_lb),
                                callbacks = early_stopping)

Epoch 1/150
30/30 [==============================] - 1s 20ms/step - loss: 0.2756 - accuracy: 0.9249 - val_loss: 0.6853 - val_accuracy: 0.7510
Epoch 2/150
30/30 [==============================] - 0s 14ms/step - loss: 0.2739 - accuracy: 0.9269 - val_loss: 0.6852 - val_accuracy: 0.7540
Epoch 3/150
30/30 [==============================] - 0s 13ms/step - loss: 0.2718 - accuracy: 0.9260 - val_loss: 0.6937 - val_accuracy: 0.7480
Epoch 4/150
30/30 [==============================] - 0s 13ms/step - loss: 0.2702 - accuracy: 0.9261 - val_loss: 0.6946 - val_accuracy: 0.7500
Epoch 5/150
30/30 [==============================] - 0s 12ms/step - loss: 0.2683 - accuracy: 0.9271 - val_loss: 0.6972 - val_accuracy: 0.7500
Epoch 6/150
30/30 [==============================] - 0s 14ms/step - loss: 0.2658 - accuracy: 0.9291 - val_loss: 0.6923 - val_accuracy: 0.7510
Epoch 7/150
30/30 [==============================] - 0s 14ms/step - loss: 0.2640 - accuracy: 0.9304 - val_loss: 0.6935 - val_accuracy: 0.7510
Epoch 

Load the best (saved) model. 

In [20]:
# Load the best (saved) model
from keras.models import load_model

saved_model = load_model('best_model.h5')

Now, use this model to to calculate the training and test accuracy: 

In [21]:
results_train = saved_model.evaluate(X_train_tokens, y_train_lb)
print(f'Training Loss: {results_train[0]:.3} \nTraining Accuracy: {results_train[1]:.3}')

print('----------')

results_test = saved_model.evaluate(X_test_tokens, y_test_lb)
print(f'Test Loss: {results_test[0]:.3} \nTest Accuracy: {results_test[1]:.3}')

235/235 [==============================] - 1s 3ms/step - loss: 0.2705 - accuracy: 0.9273
Training Loss: 0.271 
Training Accuracy: 0.927
----------
47/47 [==============================] - 0s 3ms/step - loss: 0.6187 - accuracy: 0.7853
Test Loss: 0.619 
Test Accuracy: 0.785


Nicely done! Did you notice that the model didn't train for all 150 epochs? You reduced your training time. 

Now, take a look at how regularization techniques can further improve your model performance. 

## L2 Regularization 

First, take a look at L2 regularization. Keras makes L2 regularization easy. Simply add the `kernel_regularizer=keras.regularizers.l2(lambda_coeff)` parameter to any model layer. The `lambda_coeff` parameter determines the strength of the regularization you wish to perform. 

- Use 2 hidden layers with 50 units in the first and 25 in the second layer, both with `'relu'` activation functions 
- Add L2 regularization to both the hidden layers with 0.005 as the `lambda_coeff` 

In [22]:
# Import regularizers
from keras import regularizers
random.seed(123)
L2_model = models.Sequential()

# Add the input and first hidden layer
L2_model.add(layers.Dense(units = 50, activation = 'relu', kernel_regularizer= regularizers.l2(.005), input_shape = (2000,)))

# Add another hidden layer
L2_model.add(layers.Dense(units = 25, activation = 'relu', kernel_regularizer= regularizers.l2(.005)))

# Add an output layer
L2_model.add(layers.Dense(7, activation='softmax'))

# Compile the model
L2_model.compile(optimizer='SGD', 
                 loss='categorical_crossentropy', 
                 metrics=['acc'])

# Train the model 
L2_model_val = L2_model.fit(X_train_tokens, 
                            y_train_lb, 
                            epochs=150, 
                            batch_size=256, 
                            validation_data=(X_val_tokens, y_val_lb))

Epoch 1/150
30/30 [==============================] - 1s 27ms/step - loss: 2.6094 - acc: 0.1503 - val_loss: 2.5946 - val_acc: 0.1560
Epoch 2/150
30/30 [==============================] - 0s 12ms/step - loss: 2.5683 - acc: 0.1953 - val_loss: 2.5662 - val_acc: 0.1900
Epoch 3/150
30/30 [==============================] - 0s 11ms/step - loss: 2.5412 - acc: 0.2284 - val_loss: 2.5425 - val_acc: 0.2130
Epoch 4/150
30/30 [==============================] - 0s 11ms/step - loss: 2.5156 - acc: 0.2504 - val_loss: 2.5193 - val_acc: 0.2260
Epoch 5/150
30/30 [==============================] - 0s 11ms/step - loss: 2.4888 - acc: 0.2685 - val_loss: 2.4947 - val_acc: 0.2550
Epoch 6/150
30/30 [==============================] - 0s 12ms/step - loss: 2.4590 - acc: 0.2909 - val_loss: 2.4666 - val_acc: 0.2740
Epoch 7/150
30/30 [==============================] - 0s 13ms/step - loss: 2.4263 - acc: 0.3249 - val_loss: 2.4354 - val_acc: 0.3040
Epoch 8/150
30/30 [==============================] - 0s 12ms/step - loss: 2.

30/30 [==============================] - 0s 12ms/step - loss: 1.1324 - acc: 0.8065 - val_loss: 1.2419 - val_acc: 0.7360
Epoch 63/150
30/30 [==============================] - 0s 12ms/step - loss: 1.1252 - acc: 0.8077 - val_loss: 1.2365 - val_acc: 0.7380
Epoch 64/150
30/30 [==============================] - 0s 11ms/step - loss: 1.1182 - acc: 0.8112 - val_loss: 1.2322 - val_acc: 0.7420
Epoch 65/150
30/30 [==============================] - 0s 11ms/step - loss: 1.1115 - acc: 0.8111 - val_loss: 1.2304 - val_acc: 0.7360
Epoch 66/150
30/30 [==============================] - 0s 11ms/step - loss: 1.1053 - acc: 0.8139 - val_loss: 1.2279 - val_acc: 0.7390
Epoch 67/150
30/30 [==============================] - 0s 10ms/step - loss: 1.0991 - acc: 0.8140 - val_loss: 1.2214 - val_acc: 0.7460
Epoch 68/150
30/30 [==============================] - 0s 12ms/step - loss: 1.0929 - acc: 0.8151 - val_loss: 1.2165 - val_acc: 0.7400
Epoch 69/150
30/30 [==============================] - 0s 12ms/step - loss: 1.0865 

30/30 [==============================] - 0s 12ms/step - loss: 0.8450 - acc: 0.8780 - val_loss: 1.0826 - val_acc: 0.7470
Epoch 124/150
30/30 [==============================] - 0s 12ms/step - loss: 0.8411 - acc: 0.8796 - val_loss: 1.0800 - val_acc: 0.7430
Epoch 125/150
30/30 [==============================] - 0s 11ms/step - loss: 0.8379 - acc: 0.8789 - val_loss: 1.0796 - val_acc: 0.7480
Epoch 126/150
30/30 [==============================] - 0s 11ms/step - loss: 0.8346 - acc: 0.8799 - val_loss: 1.0778 - val_acc: 0.7450
Epoch 127/150
30/30 [==============================] - 0s 12ms/step - loss: 0.8313 - acc: 0.8805 - val_loss: 1.0735 - val_acc: 0.7480
Epoch 128/150
30/30 [==============================] - 0s 12ms/step - loss: 0.8279 - acc: 0.8825 - val_loss: 1.0747 - val_acc: 0.7430
Epoch 129/150
30/30 [==============================] - 0s 11ms/step - loss: 0.8246 - acc: 0.8843 - val_loss: 1.0740 - val_acc: 0.7450
Epoch 130/150
30/30 [==============================] - 0s 12ms/step - loss: 

Now, look at the training as well as the validation accuracy for both the L2 and the baseline models. 

In [ ]:
# L2 model details
L2_model_dict = L2_model_val.history
L2_acc_values = L2_model_dict['acc'] 
L2_val_acc_values = L2_model_dict['val_acc']

# Baseline model
baseline_model_acc = baseline_model_val_dict['acc'] 
baseline_model_val_acc = baseline_model_val_dict['val_acc']

# Plot the accuracy for these models
fig, ax = plt.subplots(figsize=(12, 8))
epochs = range(1, len(acc_values) + 1)
ax.plot(epochs, L2_acc_values, label='Training acc L2')
ax.plot(epochs, L2_val_acc_values, label='Validation acc L2')
ax.plot(epochs, baseline_model_acc, label='Training acc')
ax.plot(epochs, baseline_model_val_acc, label='Validation acc')
ax.set_title('Training & validation accuracy L2 vs regular')
ax.set_xlabel('Epochs')
ax.set_ylabel('Accuracy')
ax.legend();

The results of L2 regularization are quite disappointing here. Notice the discrepancy between validation and training accuracy seems to have decreased slightly, but the end result is definitely not getting better.  


## L1 Regularization

Now have a look at L1 regularization. Will this work better? 

- Use 2 hidden layers with 50 units in the first and 25 in the second layer, both with `'relu'` activation functions 
- Add L1 regularization to both the hidden layers with 0.005 as the `lambda_coeff` 

In [23]:
random.seed(123)
L1_model = models.Sequential()

# Add the input and first hidden layer
L1_model.add(layers.Dense(units = 50, activation = 'relu', kernel_regularizer= regularizers.l1(.005), input_shape = (2000,)))


# Add a hidden layer
L1_model.add(layers.Dense(units = 25, activation = 'relu', kernel_regularizer= regularizers.l1(.005)))

# Add an output layer
L1_model.add(layers.Dense(7, activation='softmax'))

# Compile the model
L1_model.compile(optimizer='SGD', 
                 loss='categorical_crossentropy', 
                 metrics=['acc'])

# Train the model 
L1_model_val = L1_model.fit(X_train_tokens, 
                            y_train_lb, 
                            epochs=150, 
                            batch_size=256, 
                            validation_data=(X_val_tokens, y_val_lb))

Epoch 1/150
30/30 [==============================] - 1s 39ms/step - loss: 15.9970 - acc: 0.1487 - val_loss: 15.5961 - val_acc: 0.1860
Epoch 2/150
30/30 [==============================] - 0s 12ms/step - loss: 15.2388 - acc: 0.1883 - val_loss: 14.8537 - val_acc: 0.2110
Epoch 3/150
30/30 [==============================] - 0s 13ms/step - loss: 14.5058 - acc: 0.2139 - val_loss: 14.1346 - val_acc: 0.2440
Epoch 4/150
30/30 [==============================] - 0s 12ms/step - loss: 13.7948 - acc: 0.2376 - val_loss: 13.4374 - val_acc: 0.2450
Epoch 5/150
30/30 [==============================] - 0s 12ms/step - loss: 13.1055 - acc: 0.2513 - val_loss: 12.7621 - val_acc: 0.2570
Epoch 6/150
30/30 [==============================] - 0s 10ms/step - loss: 12.4379 - acc: 0.2640 - val_loss: 12.1075 - val_acc: 0.2630
Epoch 7/150
30/30 [==============================] - 0s 11ms/step - loss: 11.7909 - acc: 0.2748 - val_loss: 11.4720 - val_acc: 0.2810
Epoch 8/150
30/30 [==============================] - 0s 12ms/s

30/30 [==============================] - 0s 11ms/step - loss: 1.7539 - acc: 0.6895 - val_loss: 1.7597 - val_acc: 0.6830
Epoch 63/150
30/30 [==============================] - 0s 11ms/step - loss: 1.7436 - acc: 0.6899 - val_loss: 1.7490 - val_acc: 0.6840
Epoch 64/150
30/30 [==============================] - 0s 12ms/step - loss: 1.7338 - acc: 0.6917 - val_loss: 1.7420 - val_acc: 0.6810
Epoch 65/150
30/30 [==============================] - 0s 12ms/step - loss: 1.7241 - acc: 0.6907 - val_loss: 1.7299 - val_acc: 0.6890
Epoch 66/150
30/30 [==============================] - 0s 11ms/step - loss: 1.7148 - acc: 0.6909 - val_loss: 1.7246 - val_acc: 0.6820
Epoch 67/150
30/30 [==============================] - 0s 11ms/step - loss: 1.7055 - acc: 0.6936 - val_loss: 1.7120 - val_acc: 0.6820
Epoch 68/150
30/30 [==============================] - 0s 12ms/step - loss: 1.6962 - acc: 0.6927 - val_loss: 1.7051 - val_acc: 0.6870
Epoch 69/150
30/30 [==============================] - 0s 10ms/step - loss: 1.6872 

30/30 [==============================] - 0s 12ms/step - loss: 1.3459 - acc: 0.7103 - val_loss: 1.3648 - val_acc: 0.6910
Epoch 124/150
30/30 [==============================] - 0s 12ms/step - loss: 1.3411 - acc: 0.7101 - val_loss: 1.3597 - val_acc: 0.6920
Epoch 125/150
30/30 [==============================] - 0s 11ms/step - loss: 1.3365 - acc: 0.7113 - val_loss: 1.3510 - val_acc: 0.6960
Epoch 126/150
30/30 [==============================] - 0s 11ms/step - loss: 1.3318 - acc: 0.7116 - val_loss: 1.3444 - val_acc: 0.6920
Epoch 127/150
30/30 [==============================] - 0s 12ms/step - loss: 1.3270 - acc: 0.7113 - val_loss: 1.3473 - val_acc: 0.7000
Epoch 128/150
30/30 [==============================] - 0s 11ms/step - loss: 1.3230 - acc: 0.7109 - val_loss: 1.3378 - val_acc: 0.6970
Epoch 129/150
30/30 [==============================] - 0s 12ms/step - loss: 1.3182 - acc: 0.7132 - val_loss: 1.3337 - val_acc: 0.6920
Epoch 130/150
30/30 [==============================] - 0s 11ms/step - loss: 

Plot the training as well as the validation accuracy for the L1 model: 

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

L1_model_dict = L1_model_val.history

acc_values = L1_model_dict['acc'] 
val_acc_values = L1_model_dict['val_acc']

epochs = range(1, len(acc_values) + 1)
ax.plot(epochs, acc_values, label='Training acc L1')
ax.plot(epochs, val_acc_values, label='Validation acc L1')
ax.set_title('Training & validation accuracy with L1 regularization')
ax.set_xlabel('Epochs')
ax.set_ylabel('Accuracy')
ax.legend();

Notice how the training and validation accuracy don't diverge as much as before. Unfortunately, the validation accuracy isn't still that good. Next, experiment with dropout regularization to see if it offers any advantages. 


## Dropout Regularization 

It's time to try another technique: applying dropout to layers. As discussed in the earlier lesson, this involves setting a certain proportion of units in each layer to zero. In the following cell: 

- Apply a dropout rate of 30% to the input layer 
- Add a first hidden layer with 50 units and `'relu'` activation 
- Apply a dropout rate of 30% to the first hidden layer 
- Add a second hidden layer with 25 units and `'relu'` activation 
- Apply a dropout rate of 30% to the second hidden layer 


In [ ]:
# ⏰ This cell may take about a minute to run
random.seed(123)
dropout_model = models.Sequential()

# Implement dropout to the input layer
# NOTE: This is where you define the number of units in the input layer
dropout_model.add(layers.Dropout(0.3, input_shape=(2000,)))

# Add the first hidden layer
dropout_model.add(layers.Dense(units = 50, activation = 'relu'))

# Implement dropout to the first hidden layer 
dropout_model.add(layers.Dropout(0.3))

# Add the second hidden layer
dropout_model.add(layers.Dense(25, activation='relu'))

# Implement dropout to the second hidden layer 
dropout_model.add(layers.Dropout(0.3))

# Add the output layer
dropout_model.add(layers.Dense(7, activation='softmax'))


# Compile the model
dropout_model.compile(optimizer='SGD', 
                      loss='categorical_crossentropy', 
                      metrics=['acc'])

# Train the model
dropout_model_val = dropout_model.fit(X_train_tokens, 
                                      y_train_lb, 
                                      epochs=150, 
                                      batch_size=256, 
                                      validation_data=(X_val_tokens, y_val_lb))

Epoch 1/150
30/30 [==============================] - 1s 44ms/step - loss: 1.9512 - acc: 0.1545 - val_loss: 1.9265 - val_acc: 0.1690
Epoch 2/150
30/30 [==============================] - 1s 26ms/step - loss: 1.9409 - acc: 0.1712 - val_loss: 1.9183 - val_acc: 0.1760
Epoch 3/150
30/30 [==============================] - 1s 26ms/step - loss: 1.9304 - acc: 0.1791 - val_loss: 1.9119 - val_acc: 0.1960
Epoch 4/150
30/30 [==============================] - 1s 25ms/step - loss: 1.9247 - acc: 0.1747 - val_loss: 1.9050 - val_acc: 0.2060
Epoch 5/150
30/30 [==============================] - 1s 28ms/step - loss: 1.9093 - acc: 0.1944 - val_loss: 1.8973 - val_acc: 0.2130
Epoch 6/150
30/30 [==============================] - 1s 23ms/step - loss: 1.9093 - acc: 0.1931 - val_loss: 1.8902 - val_acc: 0.2190
Epoch 7/150
30/30 [==============================] - 1s 23ms/step - loss: 1.8966 - acc: 0.2092 - val_loss: 1.8814 - val_acc: 0.2290
Epoch 8/150
30/30 [==============================] - 1s 25ms/step - loss: 1.

In [ ]:
results_train = dropout_model.evaluate(X_train_tokens, y_train_lb)
print(f'Training Loss: {results_train[0]:.3} \nTraining Accuracy: {results_train[1]:.3}')

print('----------')

results_test = dropout_model.evaluate(X_test_tokens, y_test_lb)
print(f'Test Loss: {results_test[0]:.3} \nTest Accuracy: {results_test[1]:.3}') 

You can see here that the validation performance has improved again, and the training and test accuracy are very close!  

## Bigger Data? 

Finally, let's examine if we can improve the model's performance just by adding more data. We've quadrapled the sample dataset from 10,000 to 40,000 observations, and all you need to do is run the code! 

In [ ]:
df_bigger_sample = df.sample(40000, random_state=123)

X = df['Consumer complaint narrative']
y = df['Product']

# Train-test split
X_train_bigger, X_test_bigger, y_train_bigger, y_test_bigger = train_test_split(X, 
                                                                                y, 
                                                                                test_size=6000, 
                                                                                random_state=42)

# Validation set
X_train_final_bigger, X_val_bigger, y_train_final_bigger, y_val_bigger = train_test_split(X_train_bigger, 
                                                                                          y_train_bigger, 
                                                                                          test_size=4000, 
                                                                                          random_state=42)


# One-hot encoding of the complaints
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(X_train_final_bigger)

X_train_tokens_bigger = tokenizer.texts_to_matrix(X_train_final_bigger, mode='binary')
X_val_tokens_bigger = tokenizer.texts_to_matrix(X_val_bigger, mode='binary')
X_test_tokens_bigger = tokenizer.texts_to_matrix(X_test_bigger, mode='binary')

# One-hot encoding of products
lb = LabelBinarizer()
lb.fit(y_train_final_bigger)

y_train_lb_bigger = to_categorical(lb.transform(y_train_final_bigger))[:, :, 1]
y_val_lb_bigger = to_categorical(lb.transform(y_val_bigger))[:, :, 1]
y_test_lb_bigger = to_categorical(lb.transform(y_test_bigger))[:, :, 1]

In [ ]:
# ⏰ This cell may take several minutes to run
random.seed(123)
bigger_data_model = models.Sequential()
bigger_data_model.add(layers.Dense(50, activation='relu', input_shape=(2000,)))
bigger_data_model.add(layers.Dense(25, activation='relu'))
bigger_data_model.add(layers.Dense(7, activation='softmax'))

bigger_data_model.compile(optimizer='SGD', 
                          loss='categorical_crossentropy', 
                          metrics=['acc'])

bigger_data_model_val = bigger_data_model.fit(X_train_tokens_bigger,  
                                              y_train_lb_bigger,  
                                              epochs=150,  
                                              batch_size=256,  
                                              validation_data=(X_val_tokens_bigger, y_val_lb_bigger))

In [ ]:
results_train = bigger_data_model.evaluate(X_train_tokens_bigger, y_train_lb_bigger)
print(f'Training Loss: {results_train[0]:.3} \nTraining Accuracy: {results_train[1]:.3}')

print('----------')

results_test = bigger_data_model.evaluate(X_val_tokens_bigger, y_val_lb_bigger)
print(f'Test Loss: {results_test[0]:.3} \nTest Accuracy: {results_test[1]:.3}')

With the same amount of epochs and no regularization technique, you were able to get both better test accuracy and loss. You can still consider early stopping, L1, L2 and dropout here. It's clear that having more data has a strong impact on model performance! 


## Additional Resources

* https://github.com/susanli2016/Machine-Learning-with-Python/blob/master/Consumer_complaints.ipynb
* https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/
* https://catalog.data.gov/dataset/consumer-complaint-database 


## Summary  

In this lesson, you built deep learning models using a validation set and used several techniques such as L2 and L1 regularization, dropout regularization, and early stopping to improve the accuracy of your models. 